Making a Mongolian Food Classifier

In [1]:
import fastai
import fastcore
print('Fast.ai version:', fastai.__version__)
print('Fastcore version:', fastcore.__version__)

Fast.ai version: 2.7.14
Fastcore version: 1.5.29


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
from fastai.vision.all import *

In [4]:
from fastai.vision.widgets import *

In [5]:
from fastdownload import download_url

In [6]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install -U duckduckgo_search

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install fastcore

Note: you may need to restart the kernel to use updated packages.


In [9]:
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    with DDGS() as ddgs:
        # generator which yields dicts with:
        # {'title','image','thumbnail','url','height','width','source'}
        search_results = ddgs.images(keywords=term)
        # grap number of max_images urls
        image_urls = [next(search_results).get("image") for _ in range(max_images)]
        # convert to L (functionally extended list class from fastai)
        return L(image_urls)

ImportError: DLL load failed while importing _wrapper: The operating system cannot run %1.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Semester 6/Deep Learning/Week 2'

In [ ]:
!rm -rf mongolian_food_types

In [ ]:
mongolian_food_types = 'бууз','хуушуур','цуйван','нийслэл салат'
path = Path('mongolian_food_types')

In [ ]:
mongolian_food_types


In [ ]:
path

In [ ]:
if not path.exists():
    path.mkdir()
    for o in mongolian_food_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        urls = search_images(f'{o} ', max_images=50)
        download_images(dest, urls=urls)

Checking if Path exists

In [ ]:
path

In [ ]:
fns = get_image_files(path)
fns

Verifying images

In [ ]:
failed = verify_images(fns)
failed

Unlinking the bad images

In [ ]:
failed.map(Path.unlink);

In [ ]:
help(verify_images)

## Making  DataLoaders Object

In [ ]:
mongolian_food_types = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = mongolian_food_types.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
dls.train.show_batch(max_n=4, nrows=1)

Showing the difference kinds of cropped images for an instance of an image

In [ ]:
mongolian_food_types = mongolian_food_types.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = mongolian_food_types.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

Augmenting the images to look at them from different angles

In [ ]:
mongolian_food_types = mongolian_food_types.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = mongolian_food_types.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

## Training the Classifier

In [ ]:
mongolian_food_types = mongolian_food_types.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = mongolian_food_types.dataloaders(path)

## Using resnet18 and having 10 fine tunes seemed to work best

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

## Most of the food classifications looked good

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

## There were some images which did not belong

In [ ]:
interp.plot_top_losses(5, nrows=1)

## Exporting the Model

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
!ls

In [ ]:
from fastai.vision.all import *

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
import anvil.server
import anvil.media

anvil.server.connect("7AGWEPHYIR5LHUS7GEB5GIPG-BXIBL2GTKQFZ3RHV")

In [ ]:
@anvil.server.callable
def classify_image(file):
    with anvil.media.TempFile(file) as f:
        img = PILImage.create(f)

    pred, pred_idx, probs = learn_inf.predict(img)

    return pred, max(probs.tolist())

In [ ]:
%ls

In [ ]:
names = [{'firstname':'bob', 'lastname':'jones'}]

In [ ]:
for x in names:
    x['fullname'] = x['firstname'] + ' ' + x['lastname']